In [16]:
from Constants import *
from positions import loadpositions
from rhoova_func import *
from scipy.stats import norm
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [18]:
tasks=loadpositions()
tasks=tasks[:4]

In [19]:
tasks_df = pd.json_normalize(tasks, sep='_')
tasks_df['currency'] = tasks_df[['fixedRateBondDefinition_currency', 'floatingBondDefinition_currency','currency']].bfill(axis=1).iloc[:, 0]
tasks_df.drop(columns=['fixedRateBondDefinition_currency', 'floatingBondDefinition_currency'], inplace=True)
columns=['trade_id', 'settlementDate', 'notional','currency', 'buySell', 'calculation_type','discountCurve']
tasks_df[columns]

,trade_id,settlementDate,notional,currency,buySell,calculation_type,discountCurve
0,FRB1001,2021-02-01,1000000,TRY,Buy,fixed_rate_bond,TRY_IRS
1,FRN1001,2021-02-01,150000,USD,Buy,floating_rate_bond,USD_IRS
2,FRB1002,2021-02-01,1000000,TRY,Buy,fixed_rate_bond,TRY_OIS
3,IRS1000,2021-02-01,150000,USD,NaN,interest_rates_swap,USD_OIS


In [4]:
riskdata = {
  "id": "PORTFOLIO1",
  "name": "PORTFOLIO 1",
  "method": "VaR",
  "valuationDate": "2021-01-28",
  "valuationCurrency": "TRY",
  "riskMethod": "DELTANORMAL",
  "horizon": 252,
  "confidenceInterval": 0.99,
  "returnType": "NONE",
  "trend": False,
  "calendar": "Turkey",
  "timeBucket": ["0D","1D","1W","2W","3W","1M","2M","3M","6M","9M","1Y","2Y","3Y","4Y",
                "5Y","6Y","7Y","8Y","9Y","10Y","12Y","15Y","20Y","25Y","30Y","40Y","50Y"],
  "fillNa": "BACKWARD",
  "maxFillNaDays": 5,
  "tasks": tasks,
  "curves": [TRY_IRS,TRY_OIS,USD_IRS,USD_OIS],
  "yieldData": yielddata.to_dict('records'),
  "marketData":marketdata.to_dict('records'),
  "volatilityData":voldata.to_dict('records'),
}

In [ ]:
result=get_result(riskdata)
result

In [ ]:
def sensitivities(data):
    bp=1/10000
    data['sensitivity'] =data['cashflow'] * (pow((1+data['rate']-bp),-1*data['timeToMatByYear'])-pow((1+data['rate']),-1*data['timeToMatByYear']))/bp
    return  data

In [ ]:
def map_volatility(sensitivity,result):
    riskfactor_vol=pd.DataFrame(pd.DataFrame(result.get("deltaRiskFactors")).T.std())
    riskfactor_vol.columns=["volatility"]
    sensitivity=sensitivity.set_index("bin")
    tmp_df=pd.merge(sensitivity,riskfactor_vol,left_index=True, right_index=True)
    tmp_df["weighted_sensitivities"]=tmp_df["sensitivity"]*tmp_df["volatility"]*tmp_df["rate"]
    return tmp_df[["weighted_sensitivities"]]

In [8]:
regulatory_rw={"TRY_IRS0D":0.000065,"TRY_IRS1D":0.000065,"TRY_IRS1W":0.000065,"TRY_IRS2W":0.000065,
               "TRY_IRS3W":0.000065,"TRY_IRS1M":0.000065,"TRY_IRS2M":0.000082,"TRY_IRS3M":0.000097,
               "TRY_IRS6M":0.000079,"TRY_IRS9M":0.000079,"TRY_IRS1Y":0.000091,"TRY_IRS2Y":0.000103,
               "TRY_IRS3Y":0.000148,"TRY_IRS4Y":0.000248,"TRY_IRS5Y":0.000314,"TRY_IRS6Y":0.000425,
               "TRY_IRS7Y":0.000486,"TRY_IRS8Y":0.000565,"TRY_IRS9Y":0.000619,"TRY_IRS10Y":0.000612,
               "TRY_IRS12Y":0.000701,"TRY_IRS15Y":0.000795,"TRY_IRS20Y":0.000879,"TRY_IRS25Y":0.000922,
               "TRY_IRS30Y":0.000891,"TRY_IRS40Y":0.001186,"TRY_IRS50Y":0.001499}
rw=pd.DataFrame([regulatory_rw]).T
rw.columns=["RW"]
def map_rw(sensitivity,rw):
    sensitivity=sensitivity.set_index("bin")
    tmp_df=pd.merge(sensitivity,rw,left_index=True, right_index=True)
    tmp_df["weighted_sensitivities"]=tmp_df["sensitivity"]*tmp_df["RW"]
    return tmp_df[["weighted_sensitivities"]]

In [9]:

"""
As we have the bucket level capital charge 
now we move to interbucket capital charge
in our case we have two buckets(TRY,USD)
final capital charge is the maximum of three gamma scenarios

"""

def calculate_capital_charge(K: np.ndarray, gamma: np.ndarray, S: np.ndarray) -> float:
    """
    Calculates the capital based on given parameters.
    
    Args:
        K (np.ndarray): Array of K values per currency.
        gamma (np.ndarray): Correlation matrix for risk factors.
        S (np.ndarray): Sensitivities per currency.
    
    Returns:
        float: Computed risk capital value.
    """
    term1 = np.sum(K ** 2)
    term2 = np.sum(np.triu(gamma * np.outer(S, S), k=1))
    return np.sqrt(term1 + 2 * term2)


In [10]:
def create_gamma_matrix(size: int, coeff: float) -> np.ndarray:
    """Generates a gamma correlation matrix of given size."""
    gamma = np.full((size, size), coeff)
    np.fill_diagonal(gamma, 1.0)
    return gamma


In [11]:
rf_corr=pd.DataFrame(result.get("riskFactorsCorr"))
all_ir_riskfactor=list(result.get("forDV01").keys())
all_currencies= [rf.split("_")[0] for rf in all_ir_riskfactor]

In [12]:
all_ir_riskfactor_lst=[]
for rf in all_ir_riskfactor:
    rf_bucket=pd.DataFrame(result.get("forDV01").get(rf))
    rf_bucket["currency"]=rf_bucket["bin"].apply(lambda x: x.split('_')[0])
    all_ir_riskfactor_lst.append(rf_bucket)
all_ir_riskfactor_df=pd.concat(all_ir_riskfactor_lst)
all_ir_riskfactor_df[all_ir_riskfactor_df["currency"]=="USD"]

,bin,cashflow,rate,timeToMatByYear,currency
0,USD_IRS0D,0.000000e+00,0.001208,0.000000,USD
1,USD_IRS1D,0.000000e+00,0.001208,0.002778,USD
2,USD_IRS1W,0.000000e+00,0.001208,0.019444,USD
3,USD_IRS2W,0.000000e+00,0.001208,0.038889,USD
4,USD_IRS3W,0.000000e+00,0.001208,0.058333,USD
5,USD_IRS1M,1.219182e+06,0.001208,0.077778,USD
6,USD_IRS2M,0.000000e+00,0.001585,0.163889,USD
7,USD_IRS3M,0.000000e+00,0.002114,0.252778,USD
8,USD_IRS6M,0.000000e+00,0.002275,0.505556,USD
9,USD_IRS9M,0.000000e+00,0.002693,0.758333,USD


In [13]:
Kb_matrix_dict={}
Sb_dict={}
for curr in all_currencies:
    correlation_scenario={}
    rf_bucket=all_ir_riskfactor_df[all_ir_riskfactor_df["currency"]==curr]
    rf_sensitivities=sensitivities(rf_bucket)
    weighted_sensitivities =map_volatility(rf_sensitivities,result)
    #reg_weighted_sensitivities=map_rw(rf_sensitivities,rw)
    rf_corr_tmp=rf_corr[list(rf_bucket["bin"])]
    rf_corr_tmp=rf_corr_tmp[rf_corr_tmp.index.isin(list(rf_bucket["bin"]))]
    ## Correlation Scenarios
    Base_scenario = rf_corr_tmp.applymap(lambda x : x*1)
    High_scenario = rf_corr_tmp.applymap(lambda x : min(x*1.25,1))
    Low_scenario = rf_corr_tmp.applymap(lambda x : max(x*2 - 1 , 0.75 * x))
    Kb_base = np.sqrt(weighted_sensitivities.T@Base_scenario@weighted_sensitivities)
    Kb_low = np.sqrt(weighted_sensitivities.T@Low_scenario@weighted_sensitivities)
    Kb_high = np.sqrt(weighted_sensitivities.T@High_scenario@weighted_sensitivities)
    #Kb_matrix = np.array([Kb_base, Kb_low, Kb_high])
    correlation_scenario["base"]= np.array(Kb_base)
    correlation_scenario["low"]=np.array(Kb_low)
    correlation_scenario["high"]=np.array(Kb_high)
    Kb_matrix_dict[curr]=correlation_scenario
    Sb_dict[curr] = list(weighted_sensitivities.sum())[0] # this is used in interbucket aggregation

In [14]:
weighted_sensitivities

,weighted_sensitivities
USD_IRS0D,0.000000
USD_IRS1D,0.000000
USD_IRS1W,0.000000
USD_IRS2W,0.000000
USD_IRS3W,0.000000
USD_IRS1M,6.138379
USD_IRS2M,0.000000
USD_IRS3M,0.000000
USD_IRS6M,0.000000
USD_IRS9M,0.000000


In [15]:
# Given data
K_values =Kb_matrix_dict
S_values= Sb_dict
gamma_coefficients = {"base": 0.5, "low": 0.375, "high": 0.625}

# Extracting K arrays
currencies = list(K_values.keys())
num_currencies = len(currencies)

K_base = np.array([K_values[currency]['base'][0][0] for currency in currencies])
K_low = np.array([K_values[currency]['low'][0][0] for currency in currencies])
K_high = np.array([K_values[currency]['high'][0][0] for currency in currencies])

S = np.array([S_values[currency] for currency in currencies])

# Gamma coefficients

gamma_base = create_gamma_matrix(num_currencies, gamma_coefficients["base"])
gamma_low = create_gamma_matrix(num_currencies, gamma_coefficients["low"])
gamma_high = create_gamma_matrix(num_currencies, gamma_coefficients["high"])


# Calculate capital charges values
RC_base = calculate_capital_charge(K_base, gamma_base, S)
RC_low = calculate_capital_charge(K_low, gamma_low, S)
RC_high = calculate_capital_charge(K_high, gamma_high, S)

print(f"Gamma: Base, Capital Charge: {RC_base}")
print(f"Gamma: Low, Capital Charge: {RC_low}")
print(f"Gamma: High, Capital Charge: {RC_high}")



Gamma: Base, Capital Charge: 3822.1483238749393
Gamma: Low, Capital Charge: 4069.0905778611454
Gamma: High, Capital Charge: 3558.1086954394273
